### 번역자: 고려대학교 수학과(12) 최시현
[AiKorea 번역 프로젝트](https://github.com/aikorea/cs231n)를 이어받아서 진행하였습니다. 기존 AiKorea에서 이미 번역되어있는 CourseNote의 경우 유사한 부분이 많습니다. <br /> 
(깨진 삽화를 추가하거나 개인적으로 번역이 매끄럽지 못하다 생각하는 부분은 수정하였습니다.) <br />
문제가 될시에는 자진 삭제하겠습니다. 오류 수정 및 내용에 대한 기타 문의 사항은 passkmla@naver.com으로 연락주시기 바랍니다.

이 장에서는 다루기 쉬운 2차원의 뉴럴 네트워크를 완성해 볼 것이다. <br />
먼저 간단한 선형 분류기를 만들고 그 다음 2-레이어의 신경망으로 확장할 것이다. <br />
앞으로 보게 되겠지만, 이 확장은 매우 간단하고 몇몇 변화만을 필요로 한다.

## Generating some data

쉽게 선형적으로 분리가능하지 않은 분류 데이터를 만들어 보자. <br /> 
아래와 같은 나선형 데이터셋을 다루어보자. 

In [ ]:
N = 100 # number of points per class
D = 2 # dimensionality
K = 3 # number of classes
X = np.zeros((N*K,D)) # data matrix (each row = single example)
y = np.zeros(N*K, dtype='uint8') # class labels
for j in xrange(K):
  ix = range(N*j,N*(j+1))
  r = np.linspace(0.0,1,N) # radius
  t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2 # theta
  X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
  y[ix] = j
# lets visualize the data:
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.show()

___

<img src="./Images/1.png" width=400 />

선형적으로 분리가 불가능한 세 개의 클래스(파랑, 빨강, 노랑)으로 구성된 나선형 데이터

___

일반적으로는 각각의 feature가 0을 평균으로 가지고 unit standard deviation을 가지도록 정규화(nomalization)을 해야하지만 <br />
여기서는 이미 -1과 1사이의 값을 가지고 있기 때문에 넘어가도록 한다.

## Training Softmax Linear Classifier

#### Initialize the parameters

먼저 이 분류 데이터 셋에 대해 소프트맥스 분류기를 학습해보자. <br />
이전 장에서 보았듯이, 소프트맥스 분류기는 선형 스코어 함수를 가지고 있고 크로스 엔트로피 손실을 사용한다. <br />
선형 분류기의 파라미터들은 각 클래스에 대하여 가중치 행렬 $W$와 bias 벡터 $b$로 구성되어있다. <br />
첫 번째로 먼저 이들 파라미터들을 임의의 숫자로 초기화 하자.

In [ ]:
# initialize parameters randomly
W = 0.01 * np.random.randn(D,K)
b = np.zeros((1,K))

D=2는 차원이고 K=3은 클래스의 수이다.

#### Compute the clas scores

이것은 선형 분류기이기 때문에, 단일 행렬 곱으로 모든 클래스의 스코어를 병렬적으로 간단하게 구할 수 있다. 

In [ ]:
# compute class scores for a linear classifier
scores = np.dot(X, W) + b

이 예제에서는 300개의 2차원 점을 가지고 있고, 이 곱셈 후에 스코어 배열은 [300x3]의 크기를 가지게 된다. 
이 때 각각의 행은 3 클래스(파랑, 빨강, 노랑)에 대응하는 클래스 스코어를 준다.

#### Compute the loss

두 번째 주요 구성요소는 Unhappiness를 계산된 클래스 스코어로 정량화하는 미분가능한 손실함수이다. <br />
직관적으로, 다른 클래스보다 올바른 클래스가 더 높은 점수를 가져야한다. <br />
이 케이스에서 손실은 낮아야 하며 그렇지 않으면 손실은 높아야한다. <br />
이 직관을 정량화 할 수 있는 많은 방법들이 있지만, 이 예제에서는 소프트맥스 분류기와 관련된 크로스 엔트로피 손실을 사용해보자. <br />
만약 $f$가 하나의 예제에 대한 클래스 스코어의 배열이라면 소프트맥스 분류기는 손실을 다음과 같이 계산한다. (여기서는 배열의 크기는 3이다) <br />

<img src="./Images/2.png" width=200 />

소프트맥스 분류기를 $f$의 모든 요소로서 이 세 클래스에 대해 (unnomalized)로그의 확률을 가진다고 해석 할 수 있다. <br />
이것들을 지수화하여 비정규화(unnomalized)된 확률을 얻고, 정규화(nomalized)하여 확률을 얻는다. <br />
그러므로 이 로그 안의 표현은 정규화(nomalized)된 확률이다. 이 표현이 어떻게 동작하는지 주목하자. <br />
이 통계량은 항상 0과 1사이이다. 올바른 클래스의 확률이 매우 작을 때(거의 0)때, 이 손실은 (양의)무한대를 향해간다. <br />
반대로 올바른 클래스의 확률이 1을 향해 갈 때, 손실은 0을 향해 간다. 왜냐하면 $log(1)=0$이기 때문이다. <br />
따라서, $L_i$로 표현된 값은 올바른 클래스의 확률이 높을 때 낮고, 올바른 클래스의 확률이 낮을 때 높다. 

또한 full 소프트맥스 분류기 손실은 학습 예제와 정규화(regularization)에 대하여 크로스 엔트로피 손실로 정의된다.

<img src="./Images/3.png" width=200 />

위에서 계산된 scores라는 배열이 주어지면, 손실을 계산할 수 있다. <br />
첫째로 확률을 얻는 방법은 간단하다.

In [ ]:
num_examples = X.shape[0]
# get unnormalized probabilities
exp_scores = np.exp(scores)
# normalize them for each example
probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

이제 각 행에 대하여 클래스의 확률을 가지는 [300x3]의 probs 배열을 얻었다. <br />
특히, 이들의 합이 1이 되도록 정규화(normalize)했기 때문에,각각의 예제에 대하여 올바른 클래스에 할당된 로그 확률에 대해 질문을 할 수 있다.

In [ ]:
corect_logprobs = -np.log(probs[range(num_examples),y])

correct_logprobs 배열은 각 예제에 대해 올바른 클래스에 할당된 확률을 나타내는 1차원 배열이다. <br />
완전 손실(full-loss)은 이들 로그 확률의 평균과 정규화(regularization) 손실이다.

In [ ]:
# compute the loss: average cross-entropy loss and regularization
data_loss = np.sum(corect_logprobs)/num_examples
reg_loss = 0.5*reg*np.sum(W*W)
loss = data_loss + reg_loss

이 코드에서 정규화(regularization) 강도 $\lambda$는 reg에 저장되어 있다. <br />
정규화(regularization) 손실에 곱해지는 0.5라는 Convenience factor에 대해서는 곧 살펴볼 것이다. <br />
이것을 계산하면(임의의 파라미터들을 갖는) 작은 임의의 초기화 가중치들은 모든 클래스에 대해 대략 1/3의 확률이 할당되어 있으므로 loss = 1.1 즉, (np.log(1.0/3))을 가진다. <br />
이제 손실을 absolute lower bound인 loss = 0에 가까워지도록 최대한 낮게 만드려고 한다. <br />
즉, 손실이 낮을수록 모든 예제에 대해 올바른 클래스가 부여되어 높은 확률을 가지게 된다.

#### Computing the Analytic Gradient with Backpropagation

이제 손실을 계산하는 방법에 대하여 알아보았고, 그것을 최소화시키려고 한다. <br />
경사 하강법을 할 것이다. 즉, 임의의 파라미터들로 시작하여(위에 보이는), 파라미터들에 대해 <br />
손실에 대한 그라디언트를 계산하고 따라서 손실을 감소시키도록 파라미터들을 바꿀것이다. <br />
(정규화(normalize)된) 확률 벡터인 중간 매개 변수 $p$를 도입하자. 하나의 예제에 대한 손실은 다음과 같다.

<img src="./Images/4.png" width=300 />

이제 $f$안의 계산된 스코어에 대하여 이 예제가 전체 목적 함수(full objective) 기여하는 손실 $L_i$를 줄이기 위해 <br /> 얼마큼 변해야 하는지를 이해하려고 한다. 다시 말해, $\frac { \partial { L }_{ i } }{ \partial { f }_{ k } } $를 얻고 싶어한다. <br /> 
손실 $L_i$는 결국 $f$에 의존하는 $p$로부터 계산된다. 그라디언트를 얻기 위해 Chain-rule을 사용한다. <br />
결국에 많은 것이 상쇄되고, 매우 간단하고 이해가능한 형태가 된다.

<img src="./Images/5.png" width=200 />

이 표현은 우아하고 단순하다. <br />
계산한 확률이 p = [0.2, 0.3, 0.5]라고 가정하고, 올바른 클래스는 가운데의 0.3의 확률을 갖는 것이라 하자. <br />
따라서 이 스코어의 미분 그라디언트는 df = [0.2, -0.7, 0.5]이다. <br />
그라디언트에 대한 이해를 생각하면, 이는 매우 직관적인 결과이다. <br />
스코어 벡터 $f$의 첫 번째와 마지막 원소의 증가는 손실의 증가를 이끌고 (양의 부호를 갖는 +0.2, +0.5 때문에) 예상했던대로 손실이 증가하여 나빠지게 된다. <br />
그러나 올바른 클래스의 스코어 증가는 손실에 음의 영향을 준다. <br />
그라디언트 -0.7은 올바른 클래스 스코어의 증가가 $L_i$ 손실의 감소를 이끈다는 것을 알려준다. 

이 모든 것이 아래의 코드에 표현되어있다. prob는 각 예제에 대한 모든 클래스의(행을 따라) 확률을 저장한 것이다. <br />
dscore라 불리는 스코어의 그라디언트를 얻기위해서, 다음 과정을 진행한다.

In [ ]:
dscores = probs
dscores[range(num_examples),y] -= 1
dscores /= num_examples

마침내 scores = np.dot(X, W) + b를 가지게 되고, scores에 대한 그라디언트를 갖게 되었다.(dscore에 저장된) <br />
이제 W 와 b에 대해 역전파를 할 수 있다.

In [ ]:
dW = np.dot(X.T, dscores)
db = np.sum(dscores, axis=0, keepdims=True)
dW += reg*W # don't forget the regularization gradient

행렬 곱 연산을 통해 backpropagation을 하면서 정규화(regularization)로부터의 기여 또한 더하게 된다. <br />
정규화(regularization) 그라디언트는 reg*W로 매우 간단한 형식이다. <br /> 
왜냐하면, 상수 0.5를 이것의 loss contribution에 사용하기 때문이다. (i.e: $\frac { d }{ dw } (\frac { 1 }{ 2 } \lambda { w }^{ 2 })=\lambda w$) <br />
이는 그라디언트의 표현을 간단하게 만들기 위한 편리한 트릭이다.

#### Perfoming a parameter update

이제 계산된 그라디언트가 있고, 모든 파라미터가 손실 함수에 얼마나 영향을 주는지 안다. <br />
이제 손실을 줄이기 위해 음의 그라디언트 방향에서 파라미터 업데이트를 수행할 것이다.

In [ ]:
# perform a parameter update
W += -step_size * dW
b += -step_size * db

#### Puttinf ia all together: Training a Softmax Classifier

소프트맥스 분류기에 대한 그라디언트 디센트를 갖는 전체 코드는 다음과 같다.

In [ ]:
#Train a Linear Classifier

# initialize parameters randomly
W = 0.01 * np.random.randn(D,K)
b = np.zeros((1,K))

# some hyperparameters
step_size = 1e-0
reg = 1e-3 # regularization strength

# gradient descent loop
num_examples = X.shape[0]
for i in xrange(200):
  
  # evaluate class scores, [N x K]
  scores = np.dot(X, W) + b 
  
  # compute the class probabilities
  exp_scores = np.exp(scores)
  probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]
  
  # compute the loss: average cross-entropy loss and regularization
  corect_logprobs = -np.log(probs[range(num_examples),y])
  data_loss = np.sum(corect_logprobs)/num_examples
  reg_loss = 0.5*reg*np.sum(W*W)
  loss = data_loss + reg_loss
  if i % 10 == 0:
    print "iteration %d: loss %f" % (i, loss)
  
  # compute the gradient on scores
  dscores = probs
  dscores[range(num_examples),y] -= 1
  dscores /= num_examples
  
  # backpropate the gradient to the parameters (W,b)
  dW = np.dot(X.T, dscores)
  db = np.sum(dscores, axis=0, keepdims=True)
  
  dW += reg*W # regularization gradient
  
  # perform a parameter update
  W += -step_size * dW
  b += -step_size * db

결과는 다음과 같다.

In [ ]:
iteration 0: loss 1.096956
iteration 10: loss 0.917265
iteration 20: loss 0.851503
iteration 30: loss 0.822336
iteration 40: loss 0.807586
iteration 50: loss 0.799448
iteration 60: loss 0.794681
iteration 70: loss 0.791764
iteration 80: loss 0.789920
iteration 90: loss 0.788726
iteration 100: loss 0.787938
iteration 110: loss 0.787409
iteration 120: loss 0.787049
iteration 130: loss 0.786803
iteration 140: loss 0.786633
iteration 150: loss 0.786514
iteration 160: loss 0.786431
iteration 170: loss 0.786373
iteration 180: loss 0.786331
iteration 190: loss 0.786302

약 190번의 반복 후에 수렴했음을 볼 수 있다. 이제 학습 데이터 셋에 대한 정확도를 계산할 수 있다.

In [ ]:
# evaluate training set accuracy
scores = np.dot(X, W) + b
predicted_class = np.argmax(scores, axis=1)
print 'training accuracy: %.2f' % (np.mean(predicted_class == y))


이것은 49%의 정확도를 출력한다. 매우 좋지 않은 결과이지만, 주어진 데이터 셋이 선형 분리 불가능하게 구성되어 있기 때문에 <br /> 
개연성있는 결과이다. 이제 learned dicision boundaries을 그릴 수 있다.

___

<img src="./Images/6.png" width=400 />

선형 분류기는 간단한 나선형 데이터 셋을 학습하는 데 실패했다.

___

## Training a Neural Network

분명히 선형 분류기는 이 데이터 셋에 적당하지 않다. 따라서 뉴럴 네트워크를 사용하여 분류기를 만들어보자. <br />
하나의 추가적인 은닉층은 이 데이터에 대해 충분할 것이다. <br />
이제 가중치들과 bias들에 대하여 각각 두 개의 집합이 필요하다. (첫 번째 레이어, 두 번째 레이어)

In [ ]:
# initialize parameters randomly
h = 100 # size of hidden layer
W = 0.01 * np.random.randn(D,h)
b = np.zeros((1,h))
W2 = 0.01 * np.random.randn(h,K)
b2 = np.zeros((1,K))

점수를 계산하기 위한 forward pass는 이제 다음과 같은 형태로 바뀐다.

In [ ]:
# evaluate class scores with a 2-layer Neural Network
hidden_layer = np.maximum(0, np.dot(X, W) + b) # note, ReLU activation
scores = np.dot(hidden_layer, W2) + b2

이전 형태와 차이점은 은닉층의 표현을 계산하고, 은닉층에 기반하여 스코어를 계산하는 여분의 코드가 추가 되었다는 것이다. <br />
0에서 activation threshold를 갖는 단순한 ReLU로 비선형성을 추가하였다는 점 역시 중요하다.

남은 모든 것은 동일하다. 이전처럼 스코어에 대해 손실을 계산하고, 이전과 동일하게 이 스코어들에 대한 그라디언트 dscores를 얻는다. <br />
그러나, 물론 이 모델의 파라미터들에서 그라디언트에 대하여 backpropagate 하는 방식은 이제 바뀐다. <br />
첫 번째로, 뉴럴 네트워크의 두 번째 신경망을 backpropagate한다. <br />
이는 X(raw data)를 hidden_layer 변수로 교체한 것을 제외하고는 소프트맥스 분류기와 같은 코드처럼 보인다.

In [ ]:
# backpropate the gradient to the parameters
# first backprop into parameters W2 and b2
dW2 = np.dot(hidden_layer.T, dscores)
db2 = np.sum(dscores, axis=0, keepdims=True)

그러나, 이전과는 다르게 아직 끝낼 수 없다. 왜나하면 hidden_layer는 그 자체로는 data와 다른 파라미터들의 함수이기 때문이다. <br />
이 변수를 통해 backpropagation을 계속 할 필요가 있다. 이들의 그라디언트는 다음과 같이 계산된다.

In [ ]:
dhidden = np.dot(dscores, W2.T)

이제 은닉층의 결과에 대한 그라디언트를 가지고 있다. 다음으로 ReLU 비선형성을 backpropagate해야 한다. <br />
이것은 ReLU가 backward pass하는 동안에 효과적으로 스위치하기 때문에 매우 쉽다. <br />
$r=max(0,x)$이기 때문에 $\frac { dr }{ dx } =1(x>0)$을 가진다. <br />
Chain-rule과 결합되어, 만약에 입력이 0보다 크다면 ReLU 유닛은 바뀌는 것 없이 그라디언트 pass를 하지만, <br />
만약 forward pass 동안 입력이 0보다 작으면 이것들은 0으로 소멸된다. 이런식으로 ReLU를 간단하게 backpropagate 할 수 있다. <br />

In [ ]:
# backprop the ReLU non-linearity
dhidden[hidden_layer <= 0] = 0

마침내 첫 번째 레이어의 가중치들과 bias에 대해 계속 진행할 수 있다.

In [ ]:
# finally into W,b
dW = np.dot(X.T, dhidden)
db = np.sum(dhidden, axis=0, keepdims=True)

이제 다 되었다. $dW, db, dW2, db2$ 그라디언트들을 가지고 파라미터 업데이트를 수행할 수 있다. <br />
나머지 모든 남은 것들은 동일하다. 이 full 코드는 매우 유사하다.

In [ ]:
# initialize parameters randomly
h = 100 # size of hidden layer
W = 0.01 * np.random.randn(D,h)
b = np.zeros((1,h))
W2 = 0.01 * np.random.randn(h,K)
b2 = np.zeros((1,K))

# some hyperparameters
step_size = 1e-0
reg = 1e-3 # regularization strength

# gradient descent loop
num_examples = X.shape[0]
for i in xrange(10000):
  
  # evaluate class scores, [N x K]
  hidden_layer = np.maximum(0, np.dot(X, W) + b) # note, ReLU activation
  scores = np.dot(hidden_layer, W2) + b2
  
  # compute the class probabilities
  exp_scores = np.exp(scores)
  probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]
  
  # compute the loss: average cross-entropy loss and regularization
  corect_logprobs = -np.log(probs[range(num_examples),y])
  data_loss = np.sum(corect_logprobs)/num_examples
  reg_loss = 0.5*reg*np.sum(W*W) + 0.5*reg*np.sum(W2*W2)
  loss = data_loss + reg_loss
  if i % 1000 == 0:
    print "iteration %d: loss %f" % (i, loss)
  
  # compute the gradient on scores
  dscores = probs
  dscores[range(num_examples),y] -= 1
  dscores /= num_examples
  
  # backpropate the gradient to the parameters
  # first backprop into parameters W2 and b2
  dW2 = np.dot(hidden_layer.T, dscores)
  db2 = np.sum(dscores, axis=0, keepdims=True)
  # next backprop into hidden layer
  dhidden = np.dot(dscores, W2.T)
  # backprop the ReLU non-linearity
  dhidden[hidden_layer <= 0] = 0
  # finally into W,b
  dW = np.dot(X.T, dhidden)
  db = np.sum(dhidden, axis=0, keepdims=True)
  
  # add regularization gradient contribution
  dW2 += reg * W2
  dW += reg * W
  
  # perform a parameter update
  W += -step_size * dW
  b += -step_size * db
  W2 += -step_size * dW2
  b2 += -step_size * db2

다음을 출력한다. 

In [ ]:
iteration 0: loss 1.098744
iteration 1000: loss 0.294946
iteration 2000: loss 0.259301
iteration 3000: loss 0.248310
iteration 4000: loss 0.246170
iteration 5000: loss 0.245649
iteration 6000: loss 0.245491
iteration 7000: loss 0.245400
iteration 8000: loss 0.245335
iteration 9000: loss 0.245292

학습 정확도는 다음과 같다.

In [ ]:
# evaluate training set accuracy
hidden_layer = np.maximum(0, np.dot(X, W) + b)
scores = np.dot(hidden_layer, W2) + b2
predicted_class = np.argmax(scores, axis=1)
print 'training accuracy: %.2f' % (np.mean(predicted_class == y))

정확도 98%를 출력한다. 이제 decision boundaries를 시각화 할 수 있다.

___

<img src="./Images/7.png" width=400 />

뉴럴 네트워크 분류기는 나선형 데이터 셋을 분류해내었다.

___

## Summary

We’ve worked with a toy 2D dataset and trained both a linear network and a 2-layer Neural Network. We saw that the change from a linear classifier to a Neural Network involves very few changes in the code. The score function changes its form (1 line of code difference), and the backpropagation changes its form (we have to perform one more round of backprop through the hidden layer to the first layer of the network).

- You may want to look at this IPython Notebook code [rendered as HTML.](http://cs.stanford.edu/people/karpathy/cs231nfiles/minimal_net.html)
- Or download the [ipynb file](http://cs.stanford.edu/people/karpathy/cs231nfiles/minimal_net.ipynb)